## Imports

Imports every library needed

In [10]:
#general imports
import pandas as pd
import numpy as np

#statsmodels for regression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

#scipy for testing
from scipy import stats

#for visualization
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

## Importing the Bike Data

Imports the normal Data needed for every KPI

In [14]:
df_bikes = pd.read_csv('../data/philadelphia_2017.csv')

df_bikes["start_time"] = pd.to_datetime(df_bikes["start_time"], dayfirst = True)

df_bikes["end_time"] = pd.to_datetime(df_bikes["end_time"], dayfirst = True)

df_bikes['ride_duration_minutes'] = df_bikes['end_time'] - df_bikes['start_time']

ride_lengths = []

for label, content in df_bikes.iterrows():
    ride_lengths.append(content["ride_duration_minutes"])

ride_lengths_np = np.array(ride_lengths)
iqr = stats.iqr(ride_lengths_np)
iqr

q1,q3 = np.percentile(ride_lengths_np, [25,99.9])

lower_range = q1 - (1.5*iqr)
upper_range = q3 + (1.5*iqr)

df_bikes.drop(df_bikes[ (df_bikes.ride_duration_minutes > upper_range) | (df_bikes.ride_duration_minutes < lower_range) ].index , inplace=True)

# Weather Data Import

In [20]:
# csv import of weather data
df_weather = pd.read_csv('../data/weather_hourly_philadelphia.csv')

# change type of "date_time" to datetime
df_weather["date_time"] = pd.to_datetime(df_weather["date_time"])

# ordered weather data by date
df_weather = df_weather.sort_values(by=['date_time'])

# deleted weather date which is not needed for Philadelphia 2017

start2017 = datetime(2016, 12, 31, 23)
end2017 = datetime(2018, 1, 1)

df_2017weather = df_weather[(df_weather["date_time"] > start2017) & (df_weather["date_time"] < end2017)]
df_2017weather_unique = df_2017weather.drop_duplicates(subset='date_time')
df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))
df_2017weather_unique = df_2017weather_unique.set_index('Order')

df_2017weather_unique.tail(50)

df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][1]

for i in df_2017weather_unique.index:
    if i == 17518:
        print("End")
    elif (df_2017weather_unique['date_time'][i] - df_2017weather_unique['date_time'][i+1]) == (df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][1]):
        df_2017weather_support = {'date_time': df_2017weather_unique['date_time'][i], 'max_temp': df_2017weather_unique['max_temp'][i], 'min_temp': df_2017weather_unique['max_temp'][i], 'precip': df_2017weather_unique['precip'][i]}
        print(df_2017weather_support)

# identify missing data and generate by using average data from former and following hour

for i in df_2017weather_unique.index:
    if i == 8231:
        print("End")
    elif (df_2017weather_unique['date_time'][i] - df_2017weather_unique['date_time'][i+1]) != (df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][2]):
        df_2017weather_support = {'date_time': (df_2017weather_unique['date_time'][i] + pd.DateOffset(hours=1)), 'max_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'min_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'precip': df_2017weather_unique['precip'][i]}
        print(df_2017weather_support)
        df_2017weather_unique = df_2017weather_unique.append(df_2017weather_support, ignore_index=True)

# new order by date

df_2017weather_unique = df_2017weather_unique.sort_values(by=['date_time'])
df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))
df_2017weather_unique = df_2017weather_unique.set_index('Order')

df_2017weather_unique.tail(20)

# completing data

for i in df_2017weather_unique.index:
    if i == 8758:
        print("End")
    elif (df_2017weather_unique['date_time'][i] - df_2017weather_unique['date_time'][i+1]) != (df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][2]):
        df_2017weather_support = {'date_time': (df_2017weather_unique['date_time'][i] + pd.DateOffset(hours=1)), 'max_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'min_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'precip': df_2017weather_unique['precip'][i]}
        print(df_2017weather_support)
        df_2017weather_unique = df_2017weather_unique.append(df_2017weather_support, ignore_index=True)
# new order by date
df_2017weather_unique = df_2017weather_unique.sort_values(by=['date_time'])
df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))
df_2017weather_unique = df_2017weather_unique.set_index('Order')
# check if data is complete
for i in df_2017weather_unique.index:
    if i == 8231:
        print("Data complete")
    elif (df_2017weather_unique['date_time'][i] - df_2017weather_unique['date_time'][i+1]) != (df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][2]):
        df_2017weather_support = {'date_time': (df_2017weather_unique['date_time'][i] + pd.DateOffset(hours=1)), 'max_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'min_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'precip': df_2017weather_unique['precip'][i]}
        print(df_2017weather_support)
        df_2017weather_unique = df_2017weather_unique.append(df_2017weather_support, ignore_index=True)
df_2017weather_unique


<ipython-input-20-b0f996bbd7f9>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))


KeyError: 8232

# Putting Together weather Data and bike Data

In [ ]:
def would_be_rounded_up(num):
    if round(num) == int(num) + 1:
        return True
    else:
        return False

def get_temperature(start_time, end_time):
    duration_of_travel = end_time-start_time
    average_timestamp = end_time - datetime.fromisoformat("2017-01-01 00:00:00") - duration_of_travel/2
    weather_row = average_timestamp.total_seconds()/3600
    as_int = int(weather_row)
    if would_be_rounded_up(weather_row):
        rows = [[df_2017weather_unique["min_temp"][as_int],df_2017weather_unique["max_temp"][as_int]], [df_2017weather_unique["min_temp"][as_int+1],df_2017weather_unique["max_temp"][as_int+1]]]
    else:
        rows = [[df_2017weather_unique["min_temp"][as_int],df_2017weather_unique["max_temp"][as_int]], [df_2017weather_unique["min_temp"][as_int-1],df_2017weather_unique["max_temp"][as_int-1]]]
    result = 0
    avg_temp = (rows[0][0] + rows[0][1]) / 2
    result += avg_temp/(weather_row - int(weather_row))
    avg_temp = (rows[1][0] + rows[1][1]) / 2
    result += avg_temp/(1 - weather_row - int(weather_row))

    return result

In [16]:
df_bikes["weather"] = df_bikes.apply(lambda x: get_temperature(x[0],x[1]))

KeyError: -1

In [17]:
df_bikes

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,ride_duration_minutes
0,2017-01-01 00:05:00,2017-01-01 00:16:00,3046,3041,5347,Indego30,2nd & Market,"Girard Station, MFL",0 days 00:11:00
1,2017-01-01 00:21:00,2017-01-01 00:57:00,3110,3054,3364,Walk-up,Del. River Trail & Penn St.,Rodin Museum,0 days 00:36:00
2,2017-01-01 00:22:00,2017-01-01 00:57:00,3110,3054,2536,Walk-up,Del. River Trail & Penn St.,Rodin Museum,0 days 00:35:00
3,2017-01-01 00:27:00,2017-01-01 00:39:00,3041,3005,5176,Indego30,"Girard Station, MFL","Welcome Park, NPS",0 days 00:12:00
4,2017-01-01 00:28:00,2017-01-01 00:36:00,3047,3124,5370,Walk-up,"Independence Mall, NPS",Race Street Pier,0 days 00:08:00
...,...,...,...,...,...,...,...,...,...
788902,2017-12-31 23:05:00,2017-12-31 23:33:00,3070,3124,3708,Indego30,"Spring Garden Station, MFL",Race Street Pier,0 days 00:28:00
788903,2017-12-31 23:11:00,2018-01-01 11:03:00,3107,3165,5117,Indego30,33rd & Reservoir,24th & Race SRT,0 days 11:52:00
788904,2017-12-31 23:18:00,2017-12-31 23:25:00,3033,3046,11933,Indego30,10th & Chestnut,2nd & Market,0 days 00:07:00
788905,2017-12-31 23:39:00,2017-12-31 23:40:00,3163,3163,6725,Indego30,25th & Locust,25th & Locust,0 days 00:01:00
